# Calculating PBLH from Radiosonde observations with four methods
see https://github.com/ThomasRieutord/MetPy/blob/add_boundarylayer_module/src/metpy/calc/boundarylayer.py

In [ ]:
import numpy as np
import panda as pd
from copy import deepcopy
from metpy.calc import potential_temperature
import metpy.calc as mpcalc
import metpy.constants as mpconsts
from metpy.units import units

In [ ]:
def smooth(val, span):
    """Function that calculates the moving average with a given span.
    The span is given in number of points on which the average is made.

    Parameters
    ----------
    val: array-like
        Array of values
    span: int
        Span of the moving average. The higher the smoother

    Returns
    -------
    smoothed_val: array-like
        Array of smoothed values

    See also
    --------
    [`bottleneck.move_mean`](https://bottleneck.readthedocs.io/en/latest/reference.html#bottleneck.move_mean), 
    [`scipy.ndimage.uniform_filter1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter1d.html#scipy.ndimage.uniform_filter1d), 
    [`pandas.DataFrame.rolling`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html)
    """
    n = len(val)
    smoothed_val = deepcopy(val)
    for i in range(n):
        smoothed_val[i] = np.nanmean(val[i - min(span, i) : i + min(span, n - i)])

    return smoothed_val

In [ ]:
def bulk_richardson_number(
    height,
    potential_temperature,
    u,
    v,
    idxfoot: int = 0,
    ustar=0 * units.meter_per_second,
):
    r"""Calculate the bulk Richardson number.

    See [VH96], eq. (3):

    .. math::   Ri = (g/\theta) * \frac{(\Delta z)(\Delta \theta)}
             {\left(\Delta u)^2 + (\Delta v)^2 + b(u_*)^2}

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    potential_temperature : `pint.Quantity`
        Potential temperature profile
    u : `pint.Quantity`
        Zonal wind profile
    v : `pint.Quantity`
        Meridional wind profile
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.

    Returns
    -------
    `pint.Quantity`
        Bulk Richardson number profile
    """
    if idxfoot == 0:
        # Force the ground level to have null wind
        Du = u
        Dv = v
    else:
        Du = u - u[idxfoot]
        Dv = v - v[idxfoot]
    
    Dtheta = potential_temperature - potential_temperature[idxfoot]
    Dz = height - height[idxfoot]

    idx0 = Du**2 + Dv**2 + ustar**2 == 0
    if idx0.sum() > 0:
        bRi = np.ones_like(Dtheta) * np.nan * units.dimensionless
        bRi[~idx0] = (
            (mpconsts.g / potential_temperature[~idx0])
            * (Dtheta[~idx0] * Dz[~idx0])
            / (Du[~idx0] ** 2 + Dv[~idx0] ** 2 + ustar**2)
        )
    else:
        bRi = (
            (mpconsts.g / potential_temperature)
            * (Dtheta * Dz)
            / (Du**2 + Dv**2 + ustar**2)
        )

    return bRi

In [ ]:
def blh_from_richardson_bulk(
    height,
    potential_temperature,
    u,
    v,
    smoothingspan: int = 10,
    idxfoot: int = 0,
    bri_threshold=0.25 * units.dimensionless,
    ustar=0.1 * units.meter_per_second,
):
    """Calculate atmospheric boundary layer height with the method of
    bulk Richardson number.

    It is the height where the bulk Richardson number exceeds a given threshold.
    Well indicated for unstable boundary layers. See [VH96, Sei00, Col14, Guo16].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    potential_temperature : `pint.Quantity`
        Potential temperature profile
    u : `pint.Quantity`
        Zonal wind profile
    v : `pint.Quantity`
        Meridional wind profile
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.
    bri_threshold : `pint.Quantity`, optional
        Threshold to exceed to get boundary layer top. Defaults to 0.25
    ustar : `pint.Quantity`, optional
        Additional friction term in [VH96]. Defaluts to 0.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    bRi = bulk_richardson_number(
        height,
        smooth(potential_temperature, smoothingspan),
        smooth(u, smoothingspan),
        smooth(v, smoothingspan),
        idxfoot=idxfoot,
        ustar=ustar,
    )

    height = height[~np.isnan(bRi)]
    bRi = bRi[~np.isnan(bRi)]

    if any(bRi > bri_threshold):
        iblh = np.where(bRi > bri_threshold)[0][0]
        blh = height[iblh]
    else:
        blh = np.nan * units.meter

    return blh

In [ ]:
def blh_from_parcel(
    height,
    potential_temperature,
    smoothingspan: int = 5,
    theta0=None,
):
    """Calculate atmospheric boundary layer height with the "parcel method"
    (or "potential temperature threshold method").

    It is the height where the potential temperature profile exceeds its
    foot value. Well indicated for unstable boundary layers. See [Sei00, HL06, Col14].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    potential_temperature : `pint.Quantity`
        Potential temperature profile
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    theta0 : `pint.Quantity`, optional
        Value of theta at the foot point (skip unstruted points or add extra term). If not provided, theta[0] is taken.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    potential_temperature = smooth(potential_temperature, smoothingspan)

    if theta0 is None:
        theta0 = potential_temperature[0]

    if any(potential_temperature > theta0):
        iblh = np.where(potential_temperature > theta0)[0][0]
        blh = height[iblh]
    else:
        blh = np.nan * units.meter

    return blh

In [ ]:
def blh_from_concentration_gradient(
    height,
    concentration_profile,
    smoothingspan: int = 5,
    idxfoot: int = 0,
):
    """Calculate atmospheric boundary layer height from a concentration
    profile (specific/relative humidity, aerosol backscatter, TKE..)

    It is the height where the gradient of the concentration profile reaches a minimum.
    Well indicated for stable boundary layers. See [Sei00, HL06, Col14].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    concentration_profile : `pint.Quantity`
        Concentration profile (specific/relative humidity, aerosol backscatter, TKE..)
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    dcdz = mpcalc.first_derivative(smooth(concentration_profile, smoothingspan), x=height)
    dcdz = dcdz[idxfoot:]
    height = height[idxfoot:]
    iblh = np.argmin(dcdz)

    return height[iblh]

In [ ]:
def blh_from_temperature_inversion(
    height,
    temperature,
    smoothingspan: int = 5,
    idxfoot: int = 0,
):
    """Calculate atmospheric boundary layer height from the inversion of
    absolute temperature gradient

    It is the height where the temperature gradient (absolute or potential) changes of sign.
    Well indicated for stable boundary layers. See [Col14].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    humidity : `pint.Quantity`
        Temperature (absolute or potential) profile
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    dTdz = mpcalc.first_derivative(smooth(temperature, smoothingspan), x=height)

    dTdz = dTdz[idxfoot:]
    height = height[idxfoot:]

    if any(dTdz * dTdz[0] < 0):
        iblh = np.where(dTdz * dTdz[0] < 0)[0][0]
        blh = height[iblh]
    else:
        blh = np.nan * units.meter

    return blh


In [ ]:
df = pd.read_csv("/media/flipken/Elements/PhD_ATTO/Ceilo_Campina/Qualitycheck_data/Radiosonde/20250225_12UTC_RS.csv")

# Clean and match column names
df = df.rename(columns=lambda x: x.strip())
df = df.dropna(subset=["P [hPa]", "T [�C]", "Geopot [m]", "Ws U [m/s]", "Ws V [m/s]", "Hu [%]"])

In [ ]:
# Variables
P = df["P [hPa]"].values * units.hPa
T = (df["T [�C]"].values + 273.15) * units.kelvin
Z = df["Geopot [m]"].values * units.meter
u = df["Ws U [m/s]"].values * units("m/s")
v = df["Ws V [m/s]"].values * units("m/s")
RH = df["Hu [%]"].values * units.percent
theta = potential_temperature(P, T)

In [ ]:
blh_richardson = blh_from_richardson_bulk(Z, theta, u, v)
blh_parcel = blh_from_parcel(Z, theta)
blh_temp_inv = blh_from_temperature_inversion(Z, T)
blh_humidity = blh_from_concentration_gradient(Z, RH)

In [ ]:
print(f"Bulk Richardson      : {blh_richardson:.0f}")
print(f"Parcel Method        : {blh_parcel:.0f}")
print(f"Temperature Inversion: {blh_temp_inv:.0f}")
print(f"RH Gradient          : {blh_humidity:.0f}")